In [1]:
import numpy as np
import pandas as pd
from lifelines import KaplanMeierFitter, CoxPHFitter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import set_config
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM, FastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel

In [2]:
data = pd.read_csv("G:/내 드라이브/대학/대외/2023/연구원/연구과제/Ensemble-Kernel/veteran.csv")
np.sum(data.isna())
df = data
df = df.astype({'prior' : 'category', 'trt' : 'category', 'celltype' : 'category'})

data_0 = df[df['status']==0]
data_1 = df[df['status']==1]

x_train_0, x_test_0, target_train_0, target_test_0 = train_test_split(data_0.drop(['time','status'], axis = 1),data_0[['time','status']], test_size = 0.5, random_state = 42)
x_train_1, x_test_1, target_train_1, target_test_1 = train_test_split(data_1.drop(['time','status'], axis = 1),data_1[['time','status']], test_size = 0.5, random_state = 42)

len(x_train_0)
len(x_test_0)

len(x_train_1)
len(x_test_1)

x_test_0 = x_test_0
target_test_0 = target_test_0

x_test_1 = x_test_1
target_test_1 = target_test_1

x_train = pd.concat([x_train_0, x_train_1])
x_test = pd.concat([x_test_0, x_test_1])[:-1]
target_train = pd.concat([target_train_0, target_train_1])
target_test = pd.concat([target_test_0, target_test_1])[:-1]

x_train['time'] = target_train['time']
x_train['status'] = target_train['status']
x_test['time'] = target_test['time']
x_test['status'] = target_test['status']

In [3]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 20 to 111
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   karno     68 non-null     int64   
 1   diagtime  68 non-null     int64   
 2   age       68 non-null     int64   
 3   prior     68 non-null     category
 4   trt       68 non-null     category
 5   celltype  68 non-null     category
 6   time      68 non-null     int64   
 7   status    68 non-null     int64   
dtypes: category(3), int64(5)
memory usage: 3.8 KB


In [9]:
def my_kernel(data):
    
    temp = [0,0,0,1,1,1]
    
    def c_o(data):
        data_matrix = np.eye(len(data))
        d = np.max(data) - np.min(data)
        for i in range(len(data)):
            for j in range(len(data)):
                data_matrix[i,j] = (d-np.abs(data.iloc[i]-data.iloc[j]))/d
        return data_matrix

    def nom(data):
        data_matrix = np.eye(len(data))
        for i in range(len(data)):
            for j in range(len(data)):
                if data.iloc[i] == data.iloc[j]:
                    data_matrix[i,j] = 1
                else:
                    data_matrix[i,j] = 0
        return data_matrix
    
    from lifelines import CoxPHFitter
    coxph = CoxPHFitter()
    coxph.fit(data, duration_col = 'time', event_col = 'status')
    coef = np.log(coxph.hazard_ratios_)

    sum_matrix = 0
    coefSum = 0

    for i in range(len(temp)):

        if temp[i] == 0:

            sum_matrix += coef[i] * c_o(data[data.columns[i]])
        
        else:

            sum_matrix += coef[i] * nom(data[data.columns[i]])

        coefSum += coef[i]
            
    return sum_matrix / coefSum

In [10]:
dt = np.dtype([('status', np.bool_), ('time', np.int64 )])
my_y_train = np.empty(shape=(len(target_train),),dtype=dt)

for i in range(len(target_train)):
    my_y_train[i]=bool(target_train.iloc[i]['status']), target_train.iloc[i]['time']
    
my_y_train

array([(False, 123), (False, 182), (False,  97), (False, 231),
       ( True,  87), ( True, 110), ( True,  35), ( True, 201),
       ( True,  84), ( True,  48), ( True, 314), ( True,  30),
       ( True,  61), ( True, 126), ( True,  13), ( True,  63),
       ( True,  33), ( True,  18), ( True,  82), ( True,  21),
       ( True, 140), ( True,  53), ( True, 278), ( True, 177),
       ( True, 164), ( True,  12), ( True, 133), ( True, 103),
       ( True, 111), ( True,  44), ( True,  31), ( True,   8),
       ( True, 156), ( True,  99), ( True,   3), ( True,   7),
       ( True,   7), ( True, 200), ( True, 467), ( True,  18),
       ( True,  15), ( True, 143), ( True, 100), ( True,  36),
       ( True,   7), ( True,  31), ( True,  45), ( True, 411),
       ( True, 216), ( True,  16), ( True, 228), ( True,  22),
       ( True,  90), ( True,  24), ( True,  20), ( True,  52),
       ( True, 357), ( True,   2), ( True,  25), ( True,  49),
       ( True, 117), ( True, 105), ( True, 389), ( True

In [11]:
dt = np.dtype([('status', np.bool_), ('time', np.int64 )])
my_y_test = np.empty(shape=(len(target_test),),dtype=dt)

for i in range(len(target_test)):
    my_y_test[i]=bool(target_test.iloc[i]['status']), target_test.iloc[i]['time']
    
my_y_test

array([(False, 103), (False,  25), (False,  87), (False, 100),
       (False,  83), ( True,  12), ( True,  10), ( True,  59),
       ( True, 287), ( True,  29), ( True, 260), ( True,   1),
       ( True,  52), ( True,  15), ( True,  18), ( True,  25),
       ( True, 139), ( True, 999), ( True, 118), ( True,  80),
       ( True,  19), ( True,  54), ( True, 283), ( True, 231),
       ( True,  13), ( True, 153), ( True,   8), ( True,  19),
       ( True, 144), ( True,  73), ( True, 132), ( True, 587),
       ( True,  30), ( True, 162), ( True,  72), ( True,  11),
       ( True,  92), ( True, 378), ( True,  56), ( True, 991),
       ( True, 186), ( True,  24), ( True,   4), ( True,   1),
       ( True, 151), ( True, 117), ( True, 112), ( True,  95),
       ( True,  52), ( True, 250), ( True,  43), ( True,  42),
       ( True,  51), ( True, 242), ( True,  21), ( True,  80),
       ( True,  20), ( True,  51), ( True,   8), ( True, 553),
       ( True,  10), ( True, 111), ( True,  25), ( True

In [12]:
from sklearn.model_selection import KFold

cv = KFold(n_splits = 5, shuffle=True, random_state=42)

In [13]:
kssvm = FastKernelSurvivalSVM(optimizer = "rbtree", kernel = 'precomputed', max_iter = 1000, tol = 1e-6, random_state = 42)
surv_matrix = my_kernel(x_train)

param_grid = {'alpha': np.linspace(0.01,1,100)}
kgcv = GridSearchCV(kssvm, param_grid,
                   n_jobs=-1, refit=True,
                   cv=cv)
kgcv = kgcv.fit(surv_matrix, my_y_train)

covariate
karno      -0.037031
diagtime   -0.006160
age         0.004350
prior      -0.000334
trt         0.409687
celltype    0.202173
Name: exp(coef), dtype: float64


c:\Python\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  self.best_estimator_.fit(X, y, **fit_params)


In [14]:
kgcv.best_score_

0.7624510651278662

In [15]:
surv_matrix_test = my_kernel(x_test)

covariate
karno      -0.036157
diagtime    0.004376
age        -0.024117
prior      -0.009847
trt        -0.025258
celltype    0.042559
Name: exp(coef), dtype: float64


In [16]:
kgcv.predict(surv_matrix_test)

array([ 9.82166761e+19, -1.71910748e+20,  1.46514643e+20,  4.85162331e+19,
       -5.12451017e+20,  5.99738120e+19, -2.90110453e+19,  2.02616448e+20,
       -7.09731935e+19,  2.76028142e+20,  2.23251484e+20,  7.62589707e+19,
       -4.68484067e+20,  7.73539685e+19,  2.76805206e+20,  2.17189927e+20,
        1.20289840e+20, -5.32106031e+19, -1.52654637e+20,  2.73115755e+20,
       -4.20454591e+20,  1.02515544e+20,  1.45445694e+20,  1.01252374e+20,
        3.03584169e+20, -6.88455922e+19, -9.05554167e+19,  2.70172751e+20,
        1.41023798e+20, -5.02646722e+20, -6.54610402e+20,  1.38366544e+20,
        1.48778363e+20, -6.22569070e+20,  6.79651669e+19, -1.70773410e+20,
       -6.59634251e+20,  3.12502571e+20, -3.36737027e+19, -7.52116378e+19,
       -5.22276392e+20,  2.09211573e+20,  2.56623092e+20,  2.99296112e+20,
        1.65548925e+20, -5.49087856e+20,  1.44950511e+20,  1.96771390e+20,
        6.58350187e+19, -4.16784725e+19,  1.00715657e+20,  8.65037332e+19,
        1.25273663e+20,  

In [17]:
kgcv.best_estimator_

FastKernelSurvivalSVM(alpha=0.23, kernel='precomputed', max_iter=1000,
                      optimizer='rbtree', random_state=42, tol=1e-06)

In [18]:
x_train_data = x_train.drop(['time','status'], axis = 1)
x_test_data = x_test.drop(['time','status'], axis = 1)

In [19]:
kssvm = FastKernelSurvivalSVM(optimizer = "rbtree", kernel = 'precomputed', max_iter = 1000, tol = 1e-6, random_state = 42)
kernel_matrix = clinical_kernel(x_train_data)
param_grid = {'alpha': np.linspace(0.01,1,100)}

kgcv_cli = GridSearchCV(kssvm, param_grid,
                   n_jobs=-1, refit=True,
                   cv=cv)
kgcv_cli = kgcv_cli.fit(kernel_matrix, my_y_train)

In [20]:
kgcv_cli.best_score_

0.7720263697527412

In [21]:
kernel_matrix_test = clinical_kernel(x_test_data)

In [22]:
kgcv_cli.predict(kernel_matrix_test)

array([-29.86900265,  -5.51474753, -17.40736819,  11.23455566,
        -7.90781063,   8.37777   ,   9.2554262 ,  20.89786707,
         0.24819695,  -0.95775275,  -6.59771629,   0.47522342,
         0.39651382, -10.90662228,  19.47769855, -14.67028602,
        -0.80600223, -24.47259674,   1.70990782,  -4.38583813,
         5.03700907,   3.91837323, -16.95713865, -26.87318927,
         0.83806511,   0.95391168,  15.35500147, -13.2417774 ,
        26.97119866,   5.18184448,  10.01360065,  -4.81804905,
        -9.69708001,  13.03063018,  17.14044246,  -1.99183893,
        16.77132486, -22.47594561, -12.34208992, -21.28489282,
        -7.09057913, -13.46386671,  13.36515519,  -3.86556409,
        15.58874996,   9.33966459, -14.66307081, -16.70787046,
         1.53575076,  -9.78879315, -24.21382195,  17.677096  ,
         9.84391911,   2.11528461,   7.59559881,  12.20628746,
        20.9098305 ,  -7.61576028, -19.37274669,  -2.44335704,
        25.65134558,  -9.95130393,   8.40706598,  19.07

In [23]:
kgcv_cli.best_estimator_

FastKernelSurvivalSVM(alpha=0.97, kernel='precomputed', max_iter=1000,
                      optimizer='rbtree', random_state=42, tol=1e-06)